# Autotranslation Groovy to JavaScript

In [3]:
def r = new Random()
def nnodes = 100
def nodes = []
def links = []

for (x in (0..nnodes)){
  nodes.add(name:"" + x, group:((int) x*7/nnodes))
}

for (x in (0..(int) nnodes*1.15)) { 
  source = x % nnodes
  target = ((int) log(1 + r.nextInt(nnodes))/log(1.3))
  value = 10.0 / (1 + abs(source - target))
  links.add(source: source, target: target, value: value*value)
}

beakerx.graph = [nodes: nodes, links: links]
OutputCell.HIDDEN

1

In [4]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }});

In [5]:
%%html
<style>
.node {
  stroke: #fff;
  stroke-width: 1.5px;
}

.link {
  stroke: #999;
  stroke-opacity: .6;
}
</style>

In [6]:
%%javascript

beakerx.displayHTML(this, '<div id="fdg"></div>');

var graph = beakerx.graph
var d3 = require(['d3'], function (d3) {
    
    var width = 600,
        height = 500;

    var color = d3.scale.category20();

    var force = d3.layout.force()
        .charge(-200)
        .linkDistance(30)
        .size([width, height]);

    var svg = d3.select("#fdg")
                .append("svg")
                .attr("width", width)
                .attr("height", height);


    force.nodes(graph.nodes)
         .links(graph.links)
         .start();

    var link = svg.selectAll(".link")
          .data(graph.links)
          .enter().append("line")
          .attr("class", "link")
          .style("stroke-width", function(d) { return Math.sqrt(d.value); });

    var node = svg.selectAll(".node")
          .data(graph.nodes)
          .enter().append("circle")
          .attr("class", "node")
          .attr("r", 10)
          .style("fill", function(d) { return color(d.group); })
          .call(force.drag);

    node.append("title")
          .text(function(d) { return d.name; });

    force.on("tick", function() {
        link.attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });

        node.attr("cx", function(d) { return d.x; })
            .attr("cy", function(d) { return d.y; });
    });
});